In [2]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

results_path = "./ML_Pipeline_Features_From_Proteins_results"
os.makedirs(results_path, exist_ok=True)

**Data**

In [2]:
# Read CSV files into pandas DataFrames
df_train  = pd.read_csv("../random_split/train.csv", index_col=0)
df_dev  = pd.read_csv("../random_split/dev.csv", index_col=0)
df_test  = pd.read_csv("../random_split/test.csv", index_col=0)

# Factorize the 'family_accession' column and create a new 'label_numeric' column with numeric labels
df_train['label_numeric'] = pd.factorize(df_train['family_accession'], sort=True)[0]
df_dev['label_numeric'] = pd.factorize(df_dev['family_accession'], sort=True)[0]
df_test['label_numeric'] = pd.factorize(df_test['family_accession'], sort=True)[0]

# Convert label columns to integer lists
y_train = df_train['label_numeric'].astype(int).tolist()
y_dev = df_dev['label_numeric'].astype(int).tolist()
y_test = df_test['label_numeric'].astype(int).tolist()

# Extract sequence as string lists
X_train = df_train['sequence'].astype(str).tolist()
X_dev = df_dev['sequence'].astype(str).tolist()
X_test = df_test['sequence'].astype(str).tolist()

**Feature engineering**

Here we will create a bunch of features which will hopefully help us discriminate each sequences into one of the 18000 classes.

In [3]:
from joblib import Parallel, delayed
from tqdm import tqdm

# Extracting the list of all amino acids
amino_acids = list(set(''.join(X_train)))
print(f'amino acids (N={len(amino_acids)}): {amino_acids}')

def convert_sequence_to_feature_vector(sequences, amino_acid_list=amino_acids, num_processes=None):
    """
    Convert a list of protein sequences into a feature matrix.

    Parameters:
    - sequences (list): List of protein sequences.
    - amino_acid_list (list): List of amino acids. Default is amino_acids.
    - num_processes (int): Number of processes for parallel processing. Default is None.

    Returns:
    - pd.DataFrame: DataFrame containing the feature matrix.
    """
    # Create a DataFrame with columns as combinations of amino acids and positions
    column_names = ["Length"] + [f"{aa}_frequency" for aa in amino_acids] + [f"{aa1}->{aa2}_frequency" for aa1 in amino_acid_list for aa2 in amino_acid_list]
    df_features = pd.DataFrame(index=range(len(sequences)), columns=column_names, dtype=int)

    with tqdm(total=len(sequences), desc="Converting sequences", unit="sequence") as pbar:
        
        def process_sequence(i, sequence):
            # length of the sequence
            n = len(sequence)
            
            # Initialize features directly in the DataFrame
            df_features.loc[i, "Length"] = n
            
            # Amino acid frequencies
            for aa1 in amino_acid_list:
                count_aa1 = sequence.count(aa1)
                df_features.loc[i, f"{aa1}_frequency"] = count_aa1 / n
                
                # Amino acid pair frequencies
                for aa2 in amino_acid_list:
                    count_aa1_aa2 = sum(1 for a, b in zip(sequence, sequence[1:]) if a == aa1 and b == aa2)
                    df_features.loc[i, f"{aa1}->{aa2}_frequency"] = count_aa1_aa2 / (count_aa1 if count_aa1 != 0 else 1)
            
            pbar.update(1)

        with Parallel(n_jobs=num_processes, prefer="threads") as parallel:
            parallel(
                delayed(process_sequence)(i, sequence)
                for i, sequence in enumerate(sequences)
            )

    return df_features


amino acids (N=25): ['W', 'Y', 'U', 'D', 'E', 'P', 'L', 'A', 'I', 'S', 'C', 'F', 'V', 'G', 'N', 'R', 'B', 'Q', 'O', 'K', 'T', 'Z', 'H', 'X', 'M']


In [4]:
num_physical_cores = os.cpu_count()
print(f"Number of physical cores: {num_physical_cores}")

Number of physical cores: 4


Creating train, dev and test feature dataframes

In [5]:
# Training dataset
if os.path.exists(Path(results_path, "train features.csv")):
    print("Importing train dataset")
    X_train_features = pd.read_csv(Path(results_path, "train features.csv"))
else:
    print("Creating train dataset")
    X_train_features = convert_sequence_to_feature_vector(X_train, num_processes=2)
    print("Saving train dataset")
    X_train_features.to_csv(Path(results_path, "train features.csv"))

# Dev dataset
if os.path.exists(Path(results_path, "dev features.csv")):
    print("Importing dev dataset")
    X_dev_features = pd.read_csv(Path(results_path, "dev features.csv"))
else:
    print("Creating dev dataset")
    X_dev_features = convert_sequence_to_feature_vector(X_dev, num_processes=2)
    print("Saving dev dataset")
    X_dev_features.to_csv(Path(results_path, "dev features.csv"))

# Test dataset
if os.path.exists(Path(results_path, "test features.csv")):
    print("Importing test dataset")
    X_test_features = pd.read_csv(Path(results_path, "test features.csv"))
else:
    print("Creating test dataset")
    X_test_features = convert_sequence_to_feature_vector(X_test, num_processes=2)
    print("Saving test dataset")
    X_test_features.to_csv(Path(results_path, "test features.csv"))

Importing train dataset


In [ ]:
# Final shapes
print(X_train_features.shape)
print(len(y_train))
print(X_dev_features.shape)
print(len(y_dev))
print(X_test_features.shape)
print(len(y_test))

(1086313, 652)
1086313
(126079, 652)
126079
(126101, 652)
126101


**Extracting a subset of the training**

The dataset being very heavy, we will try to subset the training set to build a model on incrementally larger portion of the training set to see how much of the data is required to achieve optimal performances.

In [ ]:
from sklearn.model_selection import train_test_split
from collections import Counter

# % of the training set we use
subset_size = 5e-2

# Find the classes represented only once
class_counts = Counter(y_train)
classes_to_remove = [cls for cls, count in class_counts.items() if count == 1]
print(f'Classes removed (N={len(classes_to_remove)}) : {classes_to_remove}')

# Filter out instances with classes represented only once
## In training set
train_mask = ~np.isin(np.array(y_train), classes_to_remove)
X_train_filtered = X_train_features[train_mask]
y_train_filtered = np.array(y_train)[train_mask]
print(f'X_train_filtered shape : {X_train_filtered.shape}')
print(f'y_test_filtered length : {len(y_train_filtered)}')

# Subsetting the training set
_, X_train_subset, _, y_train_subset = train_test_split(
    X_train_filtered, y_train_filtered, test_size=subset_size, stratify=y_train_filtered, random_state=42
)
print(f'X_train_subset.shape : {X_train_subset.shape}')
print(f'y_train_subset.shape : {y_train_subset.shape}')

## Filtering out labels not represented in train which remain in dev
dev_mask = ~np.isin(np.array(y_dev), list(set(y_dev)-set(y_train_subset)))
X_dev_filtered = X_dev_features[dev_mask]
y_dev_filtered = np.array(y_dev)[dev_mask]
print(f'X_dev_filtered shape : {X_dev_filtered.shape}')
print(f'y_dev_filtered length : {len(y_dev_filtered)}')

## Filtering out labels not represented in train which remain in dev
test_mask = ~np.isin(np.array(y_test), list(set(y_test)-set(y_train_subset)))
X_test_filtered = X_test_features[test_mask]
y_test_filtered = np.array(y_test)[test_mask]
print(f'X_test_filtered shape : {X_test_filtered.shape}')
print(f'y_test_filtered length : {len(y_test_filtered)}')

Classes removed (N=515) : [10920, 15732, 2691, 15451, 4913, 16756, 15919, 5011, 6054, 10261, 16125, 4818, 3422, 16651, 8423, 14984, 6817, 15878, 16626, 10873, 15308, 17161, 16652, 16688, 11391, 16645, 16737, 10088, 17357, 10601, 16665, 16684, 16692, 7656, 16625, 11392, 4415, 16697, 1461, 17413, 5437, 2597, 15771, 10710, 6049, 5415, 16611, 8621, 16649, 14727, 15332, 10783, 15501, 7449, 8772, 12102, 5324, 16613, 11559, 5164, 12128, 8501, 8715, 7817, 6139, 5020, 16535, 16658, 7467, 9709, 16765, 16655, 8896, 5137, 10637, 12192, 7236, 10345, 6868, 16702, 11555, 2141, 11298, 3172, 17777, 17042, 10823, 6654, 3131, 8597, 10082, 10320, 12297, 6196, 13437, 16638, 10821, 15053, 6752, 10452, 9708, 12311, 2831, 15026, 1371, 12373, 11601, 10824, 2943, 8927, 811, 2330, 6083, 16762, 3346, 11209, 3614, 10696, 4968, 16639, 8064, 16757, 3846, 4505, 7564, 16653, 1452, 8440, 17412, 6085, 16671, 1633, 861, 16654, 15774, 17055, 16764, 5162, 16596, 16773, 2223, 17578, 5857, 16936, 1904, 15165, 15185, 17331, 4

**Preprocessing**

Here we simply remove the columns which have no variance (columns which are filled only with 1 or only with 0)

In [ ]:
# Preprocessing pipeline
preprocessing = Pipeline(
    steps=[
        ("variance", VarianceThreshold(0.)),
        ("standardize", StandardScaler())
    ]
)

# Preprocessing of X_train
X_train_std = pd.DataFrame(
                data=preprocessing.fit_transform(X_train_subset),
                index=X_train_subset.index,
                columns=preprocessing.get_feature_names_out()
            )

# Applying the same transformation to the dev dataframe
X_dev_std = pd.DataFrame(
                data=preprocessing.transform(X_dev_filtered),
                index=X_dev_filtered.index,
                columns=preprocessing.get_feature_names_out()
            )

In [ ]:
# Final shapes
print(X_train_std.shape)
print(len(y_train_subset))
print(X_dev_std.shape)
print(len(y_dev_filtered))

(54290, 470)
54290
(123003, 470)
123003


**Models**

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn import clone
import pickle

# Initializing models
logit = LogisticRegression(penalty=None,  solver ='saga', class_weight="balanced", max_iter=int(1e6), random_state=42)
logit_lasso = LogisticRegression(penalty='l1',  solver ='saga', class_weight="balanced", max_iter=int(1e6), random_state=42)
logit_ridge = LogisticRegression(penalty='l2',  solver ='saga', class_weight="balanced", max_iter=int(1e6), random_state=42)
xgbc = XGBClassifier()
rfc = RandomForestClassifier()

Fitting models

In [ ]:
f1_scores = {}

In [ ]:
model_logit = clone(logit).fit(X_train_std, y_train_subset)
f1_scores['logit'] = f1_score(y_dev_filtered, model_logit.predict(X_dev_std), average = "weighted")
print(f'logit weighted F1-score : {f1_scores["logit"]}')
# Save the trained model to a file
filename = Path(results_path, f'subsample_{subset_size}', 'logit_regression_model.sav')
pickle.dump(model_logit, open(filename, 'wb'))

In [ ]:
model_logit_lasso = clone(logit_lasso).fit(X_train_std, y_train_subset)
f1_scores['logit_lasso'] = f1_score(y_dev_filtered, model_logit_lasso.predict(X_dev_std), average = "weighted")
print(f'logit_lasso weighted F1-score : {f1_scores["logit_lasso"]}')
# Save the trained model to a file
filename = Path(results_path, f'subsample_{subset_size}', 'logit_lasso_model.sav')
pickle.dump(model_logit_lasso, open(filename, 'wb'))

In [ ]:
model_logit_ridge = clone(logit_ridge).fit(X_train_std, y_train_subset)
f1_scores['logit_ridge'] = f1_score(y_dev_filtered, model_logit_ridge.predict(X_dev_std), average = "weighted")
print(f'logit_ridge weighted F1-score : {f1_scores["logit_ridge"]}')
# Save the trained model to a file
filename = Path(results_path, f'subsample_{subset_size}', 'logit_ridge_model.sav')
pickle.dump(model_logit_ridge, open(filename, 'wb'))

In [ ]:
model_xgbc = XGBClassifier().fit(X_train_std, y_train_subset)
f1_scores['xgbc'] = f1_score(y_dev_filtered, model_xgbc.predict(X_dev_std), average = "weighted")
print(f'model_xgbc weighted F1-score : {f1_scores["xgbc"]}')

In [ ]:
model_rfc = RandomForestClassifier().fit(X_train_std, y_train_subset)
f1_scores['rfc'] = f1_score(y_dev_filtered, model_rfc.predict(X_dev_std), average = "weighted")
print(f'model_rfc weighted F1-score : {f1_scores["rfc"]}')

In [ ]:
print(f1_scores)
pd.DataFrame(f1_scores).to_csv(Path(results_path, f'subsample_{subset_size}', 'f1_scores.csv'))